Kiểm tra tổng thể csv (Minh truc meeyland)

In [66]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [67]:
def inspect_csv(df):
    print("===== CSV OVERVIEW =====")
    print(f"Rows    : {df.shape[0]}")
    print(f"Columns : {df.shape[1]}")
    print("\n--- Data types ---")
    print(df.dtypes)
    print("\n--- Memory usage ---")
    print(round(df.memory_usage(deep=True).sum() / 1024**2, 2), "MB")


In [68]:
df_raw = pd.read_csv("../data/raw/meeyland_hcm_total.csv")
inspect_csv(df_raw)


===== CSV OVERVIEW =====
Rows    : 7521
Columns : 22

--- Data types ---
id                    int64
Page                  int64
Title                object
Price_Raw            object
Price_Billion       float64
Price_per_m2         object
Area_m2             float64
District             object
Address              object
Bedrooms            float64
Toilets             float64
Post_Time            object
Link                 object
Description          object
Mặt tiền             object
Chiều sâu            object
Mức độ giao dịch     object
Ưu điểm BĐS          object
Hướng ban công       object
Tiện ích             object
Nội thất             object
Số ban công         float64
dtype: object

--- Memory usage ---
18.07 MB


Hàm kiểm tra & THÔNG BÁO số ô thiếu dữ liệu theo từng cột

In [69]:
def report_missing_values(df):
    print("===== MISSING VALUE REPORT =====")
    missing = df.isna().sum()
    percent = (missing / len(df)) * 100
    report = pd.DataFrame({
        "Missing_Count": missing,
        "Percent_Missing (%)": percent.round(2)
    }).sort_values(by="Missing_Count", ascending=False)
    print(report)
    return report


In [70]:
report_missing_values(df_raw)

===== MISSING VALUE REPORT =====
                  Missing_Count  Percent_Missing (%)
Hướng ban công             7200                95.73
Số ban công                7166                95.28
Nội thất                   7155                95.13
Tiện ích                   6828                90.79
District                   6055                80.51
Chiều sâu                  4563                60.67
Toilets                    4166                55.39
Bedrooms                   3686                49.01
Mức độ giao dịch           3524                46.86
Ưu điểm BĐS                3514                46.72
Mặt tiền                   2998                39.86
Price_Billion               808                10.74
Price_Raw                   808                10.74
Price_per_m2                774                10.29
Post_Time                   559                 7.43
Description                 559                 7.43
Area_m2                     559                 7.43
Address      

,Missing_Count,Percent_Missing (%)
Hướng ban công,7200,95.73
Số ban công,7166,95.28
Nội thất,7155,95.13
Tiện ích,6828,90.79
District,6055,80.51
Chiều sâu,4563,60.67
Toilets,4166,55.39
Bedrooms,3686,49.01
Mức độ giao dịch,3524,46.86
Ưu điểm BĐS,3514,46.72


In [71]:
mask = df_raw['Post_Time'].isna() & df_raw['Description'].isna() & df_raw['Area_m2'].isna()

mask.sum(), len(df_raw)


(np.int64(559), 7521)

Hàm chuyển đổi Price

In [72]:
def parse_price_raw(price_raw):
    if pd.isna(price_raw):
        return np.nan
    text = str(price_raw).lower().strip()

    match = re.search(r'([\d\.,]+)\s*(tỷ|ty)', text)
    if match:
        return float(match.group(1).replace(',', '.'))

    match = re.search(r'([\d\.,]+)\s*(triệu|trieu)', text)
    if match:
        return float(match.group(1).replace(',', '.')) / 1000

    return np.nan


In [73]:
def clean_price(df):
    print("===== PRICE CLEANING =====")

    mask = df['Price_Billion'].isna() & df['Price_Raw'].notna()
    df.loc[mask, 'Price_Billion'] = df.loc[mask, 'Price_Raw'].apply(parse_price_raw)

    mask = df['Price_Raw'].isna() & df['Price_Billion'].notna()
    df.loc[mask, 'Price_Raw'] = df.loc[mask, 'Price_Billion'] \
        .apply(lambda x: f"{x:.2f}".replace('.', ',') + " tỷ")

    before = len(df)
    df = df.dropna(subset=['Price_Raw', 'Price_Billion'], how='all')
    print(f"Dropped rows (missing both prices): {before - len(df)}")

    return df


Hàm extract diện tích từ text

In [74]:
def extract_area_from_text(text):
    if pd.isna(text):
        return np.nan
    text = str(text).lower()
    match = re.search(r'(\d{2,4})\s*(m2|m²)', text)
    if match:
        return float(match.group(1))
    return np.nan


In [75]:
def clean_area(df):
    print("===== AREA CLEANING =====")

    mask = df['Area_m2'].isna() & df['Title'].notna()
    df.loc[mask, 'Area_m2'] = df.loc[mask, 'Title'].apply(extract_area_from_text)

    mask = df['Area_m2'].isna() & df['Description'].notna()
    df.loc[mask, 'Area_m2'] = df.loc[mask, 'Description'].apply(extract_area_from_text)

    before = len(df)
    df = df.dropna(subset=['Area_m2'])
    print(f"Dropped rows (missing Area): {before - len(df)}")

    df = df[(df['Area_m2'] > 10) & (df['Area_m2'] < 1000)]
    return df


Kiểm tra tình trạng thiếu Bedrooms & Toilets

Hàm extract Bedrooms từ text

In [76]:
def extract_bedrooms_from_text(text):
    if pd.isna(text):
        return np.nan
    text = str(text).lower()

    patterns = [
        r'(\d+)\s*(phòng ngủ|phong ngu)',
        r'(\d+)\s*pn\b',
        r'pn\s*[:\-]?\s*(\d+)',
        r'(\d+)\s*(bedroom|br)\b'
    ]

    for p in patterns:
        m = re.search(p, text)
        if m:
            return float(m.group(1))
    return np.nan


In [77]:
def clean_bedrooms(df):
    print("===== BEDROOM CLEANING =====")
    df['Bedrooms'] = pd.to_numeric(df['Bedrooms'], errors='coerce')

    mask = df['Bedrooms'].isna() & df['Title'].notna()
    df.loc[mask, 'Bedrooms'] = df.loc[mask, 'Title'].apply(extract_bedrooms_from_text)

    mask = df['Bedrooms'].isna() & df['Description'].notna()
    df.loc[mask, 'Bedrooms'] = df.loc[mask, 'Description'].apply(extract_bedrooms_from_text)

    return df


Hàm extract Toilets từ text

In [78]:
def extract_toilets_from_text(text):
    if pd.isna(text):
        return np.nan
    text = str(text).lower()

    patterns = [
        r'(\d+)\s*(wc|toilet|toilets)',
        r'(\d+)\s*(nhà vệ sinh|nha ve sinh)',
        r'(\d+)\s*(phòng vệ sinh|phong ve sinh)',
        r'wc\s*[:\-]?\s*(\d+)',
        r'vs\s*[:\-]?\s*(\d+)'
    ]

    for p in patterns:
        m = re.search(p, text)
        if m:
            return float(m.group(1))
    return np.nan


In [79]:
def clean_toilets(df):
    print("===== TOILETS CLEANING =====")
    df['Toilets'] = pd.to_numeric(df['Toilets'], errors='coerce')

    mask = df['Toilets'].isna() & df['Title'].notna()
    df.loc[mask, 'Toilets'] = df.loc[mask, 'Title'].apply(extract_toilets_from_text)

    mask = df['Toilets'].isna() & df['Description'].notna()
    df.loc[mask, 'Toilets'] = df.loc[mask, 'Description'].apply(extract_toilets_from_text)

    return df


In [80]:
def check_bedroom_toilet_missing(df):
    b = df['Bedrooms'].isna()
    t = df['Toilets'].isna()

    print("===== BEDROOM / TOILET STATUS =====")
    print("Missing BOTH     :", (b & t).sum())
    print("Missing ONLY ONE :", (b ^ t).sum())
    print("Complete BOTH    :", (~b & ~t).sum())


KNN bedroom và toilet

In [81]:
def parse_price_per_m2(x):
    if pd.isna(x):
        return np.nan

    x = str(x).lower()

    num = re.findall(r'[\d,.]+', x)
    if not num:
        return np.nan

    value = num[0].replace(',', '.')
    try:
        value = float(value)
    except:
        return np.nan

    # Quy đổi về triệu / m2
    if 'tỷ' in x:
        return value * 1000
    elif 'triệu' in x:
        return value
    else:
        return np.nan

In [82]:
knn_features = [
    'Area_m2',
    'Price_Billion',
    'Bedrooms',
    'Toilets'
]


knn_df = df[knn_features].copy()
knn_df = knn_df.apply(pd.to_numeric, errors='coerce')

In [83]:
knn_df = df[knn_features].copy()
knn_df = knn_df.apply(pd.to_numeric, errors='coerce')


In [84]:
knn_df = knn_df.dropna(
    how='all',
    subset=['Area_m2', 'Price_Billion']
)


In [85]:
scaler = StandardScaler()
knn_scaled = scaler.fit_transform(knn_df)


In [86]:
imputer = KNNImputer(
    n_neighbors=5,
    weights='distance'
)

knn_imputed = imputer.fit_transform(knn_scaled)
knn_imputed = scaler.inverse_transform(knn_imputed)


In [87]:
knn_imputed_df = pd.DataFrame(
    knn_imputed,
    columns=knn_features,
    index=knn_df.index
)


In [88]:
df.loc[df['Bedrooms'].isna(), 'Bedrooms'] = (
    knn_imputed_df.loc[df['Bedrooms'].isna(), 'Bedrooms']
    .round()
    .clip(lower=0, upper=10)
)

df.loc[df['Toilets'].isna(), 'Toilets'] = (
    knn_imputed_df.loc[df['Toilets'].isna(), 'Toilets']
    .round()
    .clip(lower=0, upper=10)
)

In [89]:
df[['Bedrooms', 'Toilets']].isna().sum()


Bedrooms    0
Toilets     0
dtype: int64

Kiểm tra datatype

Drop row thiếu Bedrooms HOẶC Toilets

In [90]:
def drop_missing_bed_toilet(df):
    print("===== DROP MISSING BEDROOM / TOILET =====")
    before = len(df)
    df = df[df['Bedrooms'].notna() & df['Toilets'].notna()].copy()
    print(f"Total BEFORE : {before}")
    print(f"Total AFTER  : {len(df)}")
    print(f"Dropped      : {before - len(df)}")
    return df


In [91]:


df = clean_price(df)
df = clean_area(df)
df = clean_bedrooms(df)
df = clean_toilets(df)

report_missing_values(df)
check_bedroom_toilet_missing(df)

df_ready = drop_missing_bed_toilet(df)


===== PRICE CLEANING =====
Dropped rows (missing both prices): 0
===== AREA CLEANING =====
Dropped rows (missing Area): 0
===== BEDROOM CLEANING =====
===== TOILETS CLEANING =====
===== MISSING VALUE REPORT =====
                  Missing_Count  Percent_Missing (%)
Hướng ban công             6253                95.16
Số ban công                6219                94.64
Nội thất                   6207                94.46
Tiện ích                   5889                89.62
District                   5255                79.97
Chiều sâu                  3728                56.73
Ưu điểm BĐS                2764                42.06
Mức độ giao dịch           2708                41.21
Mặt tiền                   2285                34.77
Price_per_m2                117                 1.78
id                            0                 0.00
Price_Raw                     0                 0.00
Title                         0                 0.00
Page                          0              

D:\Temp\ipykernel_41908\3913130266.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'Bedrooms'] = df.loc[mask, 'Title'].apply(extract_bedrooms_from_text)
D:\Temp\ipykernel_41908\2934929757.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'Toilets'] = df.loc[mask, 'Title'].apply(extract_toilets_from_text)


DATASET CUỐI

In [92]:
print("FINAL ROWS:", len(df_ready))
df_ready.head()


FINAL ROWS: 6571


,id,Page,Title,Price_Raw,Price_Billion,Price_per_m2,Area_m2,District,Address,Bedrooms,...,Link,Description,Mặt tiền,Chiều sâu,Mức độ giao dịch,Ưu điểm BĐS,Hướng ban công,Tiện ích,Nội thất,Số ban công
0,306038613,1,"Siêu phẩm nhà mặt tiền đường Số 36 Tên Lửa, sở...","7,8 tỷ",7.8,162.50,48.0,NaN,"Q. Bình Tân, Tp. Hồ Chí Minh",4.0,...,https://meeyland.com/ban-nha-mat-pho-binh-tan-...,"Siêu phẩm nhà mặt tiền đường Số 36 Tên Lửa, sở...",4m,12m,Càng sớm càng tốt,Kinh doanh được,NaN,NaN,NaN,NaN
1,306041109,1,Tọa lạc tại vị trí đắc địa trên đường Cách Mạn...,5 tỷ,5.0,148.81,33.6,NaN,"Q. 3, Tp. Hồ Chí Minh",3.0,...,https://meeyland.com/ban-nha-rieng-quan-3-ho-c...,Tọa lạc tại vị trí đắc địa trên đường Cách Mạn...,"4,2m",8m,Càng sớm càng tốt,Giao thông thuận lợi,NaN,NaN,NaN,NaN
2,306041245,1,Nhà 2 tầng tọa lạc tại vị trí đắc địa ngay cạn...,"3,88 tỷ",3.88,71.85,54.0,NaN,"Đ. Đất Mới, Q. Bình Tân, Tp. Hồ Chí Minh",2.0,...,https://meeyland.com/ban-nha-rieng-binh-tan-ho...,Tọa lạc tại vị trí đắc địa ngay cạnh Chợ Lê Vă...,4m,"13,5m",Càng sớm càng tốt,"Kinh doanh được, Giao thông thuận lợi",NaN,NaN,NaN,NaN
3,104652329,1,"Bán căn hộ dịch vụ, hạ 1 tỷ gần Phan Văn T...",21 tỷ,21.0,175.00,120.0,NaN,"Đ. Phan Văn Trị, P. 12, Q. Bình Thạnh, Tp. Hồ ...",22.0,...,https://meeyland.com/ban-nha-mat-pho-binh-than...,"Bán căn hộ dịch vụ, hạ 1 tỷ gần Phan Văn T...","4,7m","25,53m",Càng sớm càng tốt,"Đang có lợi nhuận cao, Dân trí cao, Giao thông...",NaN,NaN,NaN,NaN
4,102855901,1,"Bán nhà hẻm ba gác, đường 14, Bình Trưng Tây, ...","5,6 tỷ",5.6,64.37,87.0,NaN,Thu gọn,2.0,...,https://meeyland.com/ban-nha-rieng-thu-duc-ho-...,"Bán nhà hẻm ba gác, đường 14, Bình Trưng Tây, ...",4m,"21,75m",Càng sớm càng tốt,"Cho thuê giá cao, Quy hoạch ổn định, Giao thôn...",Tây,NaN,NaN,NaN


Cleaning Address

In [93]:
def extract_district_final(address):
    if pd.isna(address):
        return np.nan

    # 1. Tiền xử lý: Xóa "xem thêm", đưa về chữ thường, nén nhiều khoảng trắng thành 1
    text = str(address).lower()
    text = text.replace("xem thêm", "")
    text = re.sub(r'\s+', ' ', text).strip()

    # 2. Xử lý đặc biệt cho Thủ Đức
    if 'thủ đức' in text:
        return 'Thành phố Thủ Đức'

    # 3. Dùng Regex để bắt Quận/Q + Số (Chấp nhận có hoặc không có khoảng trắng ở giữa)
    # \s* đại diện cho việc có 0 hoặc nhiều khoảng trắng
    # ([0-9]{1,2}) bắt 1 hoặc 2 chữ số
    match_num = re.search(r'(?:quận|q)\s*([0-9]{1,2})', text)
    if match_num:
        num = int(match_num.group(1)) # Ép kiểu int để "01" thành "1"
        return f"Quận {num}"

    # 4. Nếu không phải quận số, kiểm tra danh sách Quận tên chữ (Whitelist)
    # Cách này an toàn nhất để tránh "Huyện Hêm", "Huyện Thạnh"
    districts_vietnamese = {
        'bình thạnh': 'Quận Bình Thạnh',
        'gò vấp': 'Quận Gò Vấp',
        'tân bình': 'Quận Tân Bình',
        'tân phú': 'Quận Tân Phú',
        'bình tân': 'Quận Bình Tân',
        'phú nhuận': 'Quận Phú Nhuận',
        'bình chánh': 'Huyện Bình Chánh',
        'hóc môn': 'Huyện Hóc Môn',
        'củ chi': 'Huyện Củ Chi',
        'nhà bè': 'Huyện Nhà Bè',
        'cần giờ': 'Huyện Cần Giờ'
    }

    for key, value in districts_vietnamese.items():
        if key in text:
            return value

    return np.nan

In [94]:
# Cập nhật lại những dòng đang bị sai hoặc thiếu
df_ready['District'] = df_ready['Address'].apply(extract_district_final)

# Kiểm tra lại 10 mẫu ngẫu nhiên
print(df_ready[['District', 'Address']].sample(10))

# Xem danh sách các Quận đã được gom nhóm
print(df_ready['District'].unique())

               District                                            Address
4077  Thành phố Thủ Đức  Đ. 385, P. Tăng Nhơn Phú A, Tp. Thủ Đức, Tp. H...
79         Huyện Củ Chi                         H. Củ Chi, Tp. Hồ Chí Minh
1829                NaN                                            Thu gọn
4634  Thành phố Thủ Đức  Đ. Lý Tế Xuyên, P. Linh Đông, Tp. Thủ Đức, Tp....
693         Quận Gò Vấp         Đ. Số 1, P. 16, Q. Gò Vấp, Tp. Hồ Chí Minh
383       Quận Bình Tân  Đ. 6, P. Bình Hưng Hoà B, Q. Bình Tân, Tp. Hồ ...
597                 NaN  Đ. Lê Văn Lương, P. Tân Hưng, Q. 7, Tp. Hồ Chí...
968                 NaN                             Q. 12, Tp. Hồ Chí Minh
7391      Quận Tân Bình                P. 15, Q. Tân Bình, Tp. Hồ Chí Minh
5548        Quận Gò Vấp  Đ. Phạm Văn Chiêu, P. 8, Q. Gò Vấp, Tp. Hồ Chí...
['Quận Bình Tân' nan 'Quận Bình Thạnh' 'Quận Tân Phú' 'Quận Gò Vấp'
 'Thành phố Thủ Đức' 'Quận Phú Nhuận' 'Quận Tân Bình' 'Huyện Hóc Môn'
 'Huyện Củ Chi' 'Huyện Bình Chánh' 'H

In [95]:

def extract_district_from_address(address):
    if pd.isna(address):
        return np.nan

    # 1. TIỀN XỬ LÝ: Đưa về chữ thường, xóa "xem thêm", nén khoảng trắng
    text = str(address).lower()
    text = text.replace("xem thêm", "")
    # Xóa các dấu chấm sau các từ viết tắt để dễ xử lý (q. -> q, h. -> h)
    text = re.sub(r'(?<=\b[qht])\.', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 2. ƯU TIÊN CAO NHẤT: Thành phố Thủ Đức
    # (Bao gồm các kiểu: tp thủ đức, thành phố thủ đức, q thủ đức...)
    if any(kw in text for kw in ['thủ đức', 'thu duc']):
        return 'Thành phố Thủ Đức'

    # 3. XỬ LÝ QUẬN SỐ (Q1, Q 1, Q 01, Quận 1, Quận 01, Q.1...)
    # Giải thích Regex: (?:quận|q) tìm 'quận' hoặc 'q'. \s* cho phép nhiều khoảng trắng.
    # ([0-9]{1,2}) bắt lấy 1 hoặc 2 chữ số.
    match_num = re.search(r'\b(?:quận|q)\s*([0-9]{1,2})\b', text)
    if match_num:
        num = int(match_num.group(1)) # Ép kiểu int để biến 01, 02 thành 1, 2
        return f"Quận {num}"

    # 4. XỬ LÝ QUẬN/HUYỆN TÊN CHỮ (Bình Thạnh, Gò Vấp, Nhà Bè...)
    # Dùng Whitelist để tránh bắt nhầm "Huyện Hêm" từ "Xem thêm"
    dist_map = {
        'bình thạnh': 'Quận Bình Thạnh',
        'gò vấp': 'Quận Gò Vấp',
        'tân bình': 'Quận Tân Bình',
        'tân phú': 'Quận Tân Phú',
        'bình tân': 'Quận Bình Tân',
        'phú nhuận': 'Quận Phú Nhuận',
        'bình chánh': 'Huyện Bình Chánh',
        'hóc môn': 'Huyện Hóc Môn',
        'củ chi': 'Huyện Củ Chi',
        'nhà bè': 'Huyện Nhà Bè',
        'cần giờ': 'Huyện Cần Giờ',
        'quận 12': 'Quận 12' # Đề phòng trường hợp quận số viết chữ
    }

    for key, value in dist_map.items():
        if key in text:
            return value

    # 5. CÁC QUẬN CÒN LẠI (Quận 1 đến Quận 11 - nếu viết kiểu chữ)
    for i in range(1, 12):
        if f'quận {i}' in text or f'q {i}' in text:
            return f"Quận {i}"

    return np.nan

def clean_project_district(df):
    print("===== BẮT ĐẦU CHUYỂN ĐỔI ADDRESS -> DISTRICT =====")

    # Tạo mask: Những dòng District đang thiếu NHƯNG Address có dữ liệu
    mask = df['District'].isna() & df['Address'].notna()
    print(f"Số dòng cần xử lý: {mask.sum()}")

    # Thực hiện chuyển đổi
    extracted_data = df.loc[mask, 'Address'].apply(extract_district_from_address)

    # Điền vào cột District
    df.loc[mask, 'District'] = extracted_data

    print("Kết quả sau khi xử lý:")
    print(df['District'].value_counts())
    print(f"Số dòng vẫn còn thiếu District: {df['District'].isna().sum()}")

    return df

# --- THỰC THI ---
# Giả sử df_ready là dataframe của bạn
df_ready = clean_project_district(df_ready)

# Kiểm tra các mẫu dữ liệu thực tế bạn đã đưa ra
test_indices = [1190, 877, 168, 1350, 644, 1167]
# Lọc những index có tồn tại trong df_ready để in ra kiểm tra
existing_indices = [i for i in test_indices if i in df_ready.index]
print("\nKIỂM TRA MẪU SAU KHI FIX:")
print(df_ready.loc[existing_indices, ['District', 'Address']])

===== BẮT ĐẦU CHUYỂN ĐỔI ADDRESS -> DISTRICT =====
Số dòng cần xử lý: 2141
Kết quả sau khi xử lý:
District
Quận Bình Thạnh      1118
Thành phố Thủ Đức     890
Quận Gò Vấp           586
Quận Tân Bình         378
Quận Phú Nhuận        371
Quận Bình Tân         348
Quận Tân Phú          322
Quận 12               270
Quận 7                257
Quận 10               212
Quận 1                211
Quận 3                177
Huyện Hóc Môn         156
Quận 8                147
Huyện Bình Chánh      111
Quận 5                 97
Quận 6                 91
Huyện Nhà Bè           76
Quận 11                70
Huyện Củ Chi           62
Quận 4                 61
Huyện Cần Giờ          11
Name: count, dtype: int64
Số dòng vẫn còn thiếu District: 549

KIỂM TRA MẪU SAU KHI FIX:
           District                                          Address
1190            NaN                                          Thu gọn
877          Quận 7      Đ. Số 53, P. Tân Quy, Q. 7, Tp. Hồ Chí Minh
168   Quận Tân Bình  Đ. C

In [96]:
# 1. Lấy danh sách các dòng đã được chỉnh sửa (District từ NaN thành có giá trị)
# Giả sử mask_processed là những dòng mà trước đó District bị thiếu
df_fixed = df_ready[df_ready['Address'].notna() & df_ready['District'].notna()].copy()

# 2. Lấy danh sách các dòng vẫn chưa thể thay đổi (Vẫn là NaN)
df_remain_nan = df_ready[df_ready['District'].isna()].copy()

print("--- THỐNG KÊ KẾT QUẢ ---")
print(f"Số dòng đã sửa đúng: {len(df_fixed)}")
print(f"Số dòng vẫn còn trống: {len(df_remain_nan)}")

# 3. Hiển thị bảng so sánh các dòng đã sửa (Top 10 dòng đầu)
print("\n===== BẢNG 1: CÁC DÒNG ĐÃ CHỈNH SỬA ĐÚNG (MẪU) =====")
if not df_fixed.empty:
    print(df_fixed[['Address', 'District']].head(10).to_markdown())
else:
    print("Không có dòng nào được chỉnh sửa.")

# 4. Hiển thị bảng các dòng chưa sửa được (Để kiểm tra lý do)
print("\n===== BẢNG 2: CÁC DÒNG CHƯA THỂ THAY ĐỔI (CẦN KIỂM TRA) =====")
if not df_remain_nan.empty:
    # Chỉ lấy cột Address để soi lỗi
    print(df_remain_nan[['Address']].head(10).to_markdown())
else:
    print("Tuyệt vời! Không còn dòng nào bị thiếu.")

--- THỐNG KÊ KẾT QUẢ ---
Số dòng đã sửa đúng: 6022
Số dòng vẫn còn trống: 549

===== BẢNG 1: CÁC DÒNG ĐÃ CHỈNH SỬA ĐÚNG (MẪU) =====
|    | Address                                                   | District          |
|---:|:----------------------------------------------------------|:------------------|
|  0 | Q. Bình Tân, Tp. Hồ Chí Minh                              | Quận Bình Tân     |
|  1 | Q. 3, Tp. Hồ Chí Minh                                     | Quận 3            |
|  2 | Đ. Đất Mới, Q. Bình Tân, Tp. Hồ Chí Minh                  | Quận Bình Tân     |
|  3 | Đ. Phan Văn Trị, P. 12, Q. Bình Thạnh, Tp. Hồ Chí Minh    | Quận Bình Thạnh   |
|  6 | Đ. Lê Đình Thám, P. Tân Quý, Q. Tân Phú, Tp. Hồ Chí Minh  | Quận Tân Phú      |
|  7 | Đ. Nguyễn Hữu Cảnh, P. 22, Q. Bình Thạnh, Tp. Hồ Chí Minh | Quận Bình Thạnh   |
|  8 | Đ. Nguyễn Cư Trinh, P. Cô Giang, Q. 1, Tp. Hồ Chí Minh    | Quận 1            |
|  9 | Đ. Phạm Văn Chiêu, P. 16, Q. Gò Vấp, Tp. Hồ Chí Minh      | Quận Gò Vấp       

In [97]:
# 'utf-8-sig' giúp Excel nhận diện đúng tiếng Việt có dấu
df_fixed.to_csv("../data/cleaned/cleanMey.csv", index=False, encoding='utf-8-sig')

In [107]:
df_fixed.head()

,id,Page,Title,Price_Raw,Price_Billion,Price_per_m2,Area_m2,District,Address,Bedrooms,...,Link,Description,Mặt tiền,Chiều sâu,Mức độ giao dịch,Ưu điểm BĐS,Hướng ban công,Tiện ích,Nội thất,Số ban công
0,306038613,1,"Siêu phẩm nhà mặt tiền đường Số 36 Tên Lửa, sở...","7,8 tỷ",7.80,162.500000,48.0,Quận Bình Tân,"Q. Bình Tân, Tp. Hồ Chí Minh",4.0,...,https://meeyland.com/ban-nha-mat-pho-binh-tan-...,"Siêu phẩm nhà mặt tiền đường Số 36 Tên Lửa, sở...",4m,12m,Càng sớm càng tốt,Kinh doanh được,NaN,NaN,NaN,NaN
1,306041109,1,Tọa lạc tại vị trí đắc địa trên đường Cách Mạn...,5 tỷ,5.00,148.809524,33.6,Quận 3,"Q. 3, Tp. Hồ Chí Minh",3.0,...,https://meeyland.com/ban-nha-rieng-quan-3-ho-c...,Tọa lạc tại vị trí đắc địa trên đường Cách Mạn...,"4,2m",8m,Càng sớm càng tốt,Giao thông thuận lợi,NaN,NaN,NaN,NaN
2,306041245,1,Nhà 2 tầng tọa lạc tại vị trí đắc địa ngay cạn...,"3,88 tỷ",3.88,71.851852,54.0,Quận Bình Tân,"Đ. Đất Mới, Q. Bình Tân, Tp. Hồ Chí Minh",2.0,...,https://meeyland.com/ban-nha-rieng-binh-tan-ho...,Tọa lạc tại vị trí đắc địa ngay cạnh Chợ Lê Vă...,4m,"13,5m",Càng sớm càng tốt,"Kinh doanh được, Giao thông thuận lợi",NaN,NaN,NaN,NaN
3,104652329,1,"Bán căn hộ dịch vụ, hạ 1 tỷ gần Phan Văn T...",21 tỷ,21.00,175.000000,120.0,Quận Bình Thạnh,"Đ. Phan Văn Trị, P. 12, Q. Bình Thạnh, Tp. Hồ ...",22.0,...,https://meeyland.com/ban-nha-mat-pho-binh-than...,"Bán căn hộ dịch vụ, hạ 1 tỷ gần Phan Văn T...","4,7m","25,53m",Càng sớm càng tốt,"Đang có lợi nhuận cao, Dân trí cao, Giao thông...",NaN,NaN,NaN,NaN
6,104652322,1,"Bán nhà mặt tiền kinh doanh Lê Đình Thám, ...","9,3 tỷ",9.30,140.909091,66.0,Quận Tân Phú,"Đ. Lê Đình Thám, P. Tân Quý, Q. Tân Phú, Tp. H...",3.0,...,https://meeyland.com/ban-nha-mat-pho-tan-phu-h...,"Bán nhà mặt tiền kinh doanh Lê Đình Thám, ...","4,2m","15,71m",Càng sớm càng tốt,"Kinh doanh được, Cho thuê giá cao, Dân trí cao",NaN,NaN,NaN,NaN


In [108]:
inspect_csv(df_fixed)

===== CSV OVERVIEW =====
Rows    : 6022
Columns : 22

--- Data types ---
id                    int64
Page                  int64
Title                object
Price_Raw            object
Price_Billion       float64
Price_per_m2        float64
Area_m2             float64
District             object
Address              object
Bedrooms             object
Toilets              object
Post_Time            object
Link                 object
Description          object
Mặt tiền             object
Chiều sâu            object
Mức độ giao dịch     object
Ưu điểm BĐS          object
Hướng ban công       object
Tiện ích             object
Nội thất             object
Số ban công         float64
dtype: object

--- Memory usage ---
14.91 MB


In [109]:
report_missing_values(df_fixed)

===== MISSING VALUE REPORT =====
                  Missing_Count  Percent_Missing (%)
Số ban công                6009                99.78
Nội thất                   5997                99.58
Hướng ban công             5861                97.33
Tiện ích                   5816                96.58
Chiều sâu                  3720                61.77
Ưu điểm BĐS                2762                45.87
Mức độ giao dịch           2692                44.70
Mặt tiền                   2281                37.88
Price_per_m2                  0                 0.00
Price_Billion                 0                 0.00
Page                          0                 0.00
id                            0                 0.00
Price_Raw                     0                 0.00
Title                         0                 0.00
Description                   0                 0.00
Link                          0                 0.00
Post_Time                     0                 0.00
Toilets      

,Missing_Count,Percent_Missing (%)
Số ban công,6009,99.78
Nội thất,5997,99.58
Hướng ban công,5861,97.33
Tiện ích,5816,96.58
Chiều sâu,3720,61.77
Ưu điểm BĐS,2762,45.87
Mức độ giao dịch,2692,44.70
Mặt tiền,2281,37.88
Price_per_m2,0,0.00
Price_Billion,0,0.00


In [110]:
# ==============================
# RE-CALCULATE PRICE PER M2
# ==============================

# đảm bảo numeric
df_fixed['Price_Billion'] = pd.to_numeric(
    df_fixed['Price_Billion'], errors='coerce'
)

df_fixed['Area_m2'] = pd.to_numeric(
    df_fixed['Area_m2'], errors='coerce'
)

# loại diện tích không hợp lệ
df_fixed.loc[df_fixed['Area_m2'] <= 0, 'Area_m2'] = np.nan

# tính giá / m2 (triệu / m2)
df_fixed['Price_per_m2'] = (
    df_fixed['Price_Billion'] * 1000
) / df_fixed['Area_m2']

# clip theo thị trường VN
df_fixed['Price_per_m2'] = df_fixed['Price_per_m2'].clip(
    lower=5,    # min hợp lý
    upper=500   # max hợp lý
)

# 🔹 LÀM TRÒN 1 CHỮ SỐ THẬP PHÂN
df_fixed['Price_per_m2'] = df_fixed['Price_per_m2'].round(1)
print("DONE: Price_per_m2 recalculated")

DONE: Price_per_m2 recalculated


In [111]:
df_fixed[['Price_Billion', 'Area_m2', 'Price_per_m2']].describe()


,Price_Billion,Area_m2,Price_per_m2
count,6022.000000,6022.000000,6022.000000
mean,15.023591,101.242012,144.494703
std,27.679838,103.944183,93.092337
min,0.300000,10.005000,5.000000
25%,5.390000,52.000000,83.300000
50%,8.500000,72.000000,126.200000
75%,14.500000,105.000000,183.300000
max,700.000000,999.000000,500.000000


In [112]:
df_fixed[
    (df_fixed['Price_Billion'] > 0) &
    (df_fixed['Area_m2'] > 0) &
    (df_fixed['Price_per_m2'].isna())
].shape


(0, 22)

In [113]:
df_fixed.to_csv("../data/cleaned/df_fixed_final.csv",encoding="utf-8-sig", index=False)


In [114]:
report_missing_values(df_fixed)

===== MISSING VALUE REPORT =====
                  Missing_Count  Percent_Missing (%)
Số ban công                6009                99.78
Nội thất                   5997                99.58
Hướng ban công             5861                97.33
Tiện ích                   5816                96.58
Chiều sâu                  3720                61.77
Ưu điểm BĐS                2762                45.87
Mức độ giao dịch           2692                44.70
Mặt tiền                   2281                37.88
Price_per_m2                  0                 0.00
Price_Billion                 0                 0.00
Page                          0                 0.00
id                            0                 0.00
Price_Raw                     0                 0.00
Title                         0                 0.00
Description                   0                 0.00
Link                          0                 0.00
Post_Time                     0                 0.00
Toilets      

,Missing_Count,Percent_Missing (%)
Số ban công,6009,99.78
Nội thất,5997,99.58
Hướng ban công,5861,97.33
Tiện ích,5816,96.58
Chiều sâu,3720,61.77
Ưu điểm BĐS,2762,45.87
Mức độ giao dịch,2692,44.70
Mặt tiền,2281,37.88
Price_per_m2,0,0.00
Price_Billion,0,0.00


In [115]:
df_fixed['Post_Time'].head(10)
df_fixed['Post_Time'].dtype


dtype('O')

In [116]:
# ==============================
# POST TIME CLEANING
# ==============================

# strip + convert về string
df_fixed['Post_Time'] = df_fixed['Post_Time'].astype(str).str.strip()

# parse datetime (tự nhận dạng format)
df_fixed['Post_Time'] = pd.to_datetime(
    df_fixed['Post_Time'],
    errors='coerce',
    dayfirst=True   # QUAN TRỌNG cho dữ liệu VN
)

print("DONE: Post_Time standardized")

DONE: Post_Time standardized


In [117]:
missing_time = df_fixed['Post_Time'].isna().sum()
total = len(df_fixed)

print(f"Post_Time missing: {missing_time} / {total} ({missing_time/total*100:.2f}%)")


Post_Time missing: 0 / 6022 (0.00%)


In [118]:
df_fixed['Post_Year']  = df_fixed['Post_Time'].dt.year
df_fixed['Post_Month'] = df_fixed['Post_Time'].dt.month
df_fixed['Post_Day']   = df_fixed['Post_Time'].dt.day
df_fixed['Post_Weekday'] = df_fixed['Post_Time'].dt.weekday


In [119]:
df_fixed.to_csv(
    "../data/cleaned/df_fixed_final.csv",
    index=False,
    encoding="utf-8-sig"
)
